## First Demo - Sorting Demo

```
specs = [
    Metadata(), 
    PottsCore(dim_x=100, dim_y=100, neighbor_order=2)
]

cell_types = ["Condensing", "NonCondensing"]
specs.append(CellTypePlugin(*cell_types))

target_volume = 25
lambda_volume = 2

volume_specs = VolumePlugin()
volume_specs.param_new(cell_types[0], target_volume=target_volume, lambda_volume=lambda_volume)
volume_specs.param_new(cell_types[1], target_volume=target_volume, lambda_volume=lambda_volume)
specs.append(volume_specs)
```

In [22]:
# Inject CSS for input styling and error states
from IPython.display import display, HTML as IPythonHTML
display(IPythonHTML("""
<style>
/* Round corners for all input boxes */
.widget-text input,
.widget-bounded-int-text input,
.widget-bounded-float-text input,
.widget-float-text input,
.widget-int-text input {
    border-radius: 4px !important;
}

/* Round corners for dropdown/select inputs */
.widget-dropdown select,
.widget-select select {
    border-radius: 4px !important;
}

/* Round corners for buttons */
.widget-button button,
.jupyter-button {
    border-radius: 4px !important;
}

/* Spacing classes for layout containers */
.vbox-row-spacing {
    margin: 10px 0 !important;
}

.hbox-item-spacing {
    margin: 0 15px 0 0 !important;
}

.vbox-no-margin {
    margin: 0 !important;
}

.hbox-no-margin {
    margin: 0 !important;
}

.celltype-item-spacing {
    margin: 0 0 5px 0 !important;
}

.small-right-spacing {
    margin: 0 5px 0 0 !important;
}

/* Plugin-specific spacing classes */
.plugin-config-container {
    padding: 0 0 0 0 !important;
}
.plugin-compact-container {
    padding: 0 !important;
    margin: 0 !important;
}
.plugin-input-spacing {
    margin: 0 15px 0 0 !important;
}
.plugin-bottom-spacing {
    margin: 0 0 15px 0 !important;
}
.plugin-checkbox-bottom-spacing {
    margin: 0 0 15px 0 !important;
}
.button-spacing {
    margin: 10px 0 !important;
}

/* Error state styling with rounded corners */
.error-input input {
    border: 2px solid #f44336 !important;
    background-color: #ffebee !important;
    box-shadow: 0 0 3px rgba(244, 67, 54, 0.3) !important;
    border-radius: 4px !important;
}

.error-input input:focus {
    border-color: #d32f2f !important;
    box-shadow: 0 0 5px rgba(244, 67, 54, 0.5) !important;
    border-radius: 4px !important;
}

.plugin-top-spacing {
    margin-top: 10px !important;
}
</style>
"""))


In [ ]:
from Spec_UI_1 import SpecificationSetupUI

# ui = SpecificationSetupUI()

# Debuging:
# + Random seed should be a number generator number (positive integer)
# + Reset celltype and reset pottscore needs to be fixed?
# Neighbor order can't be negative for plugins

# Newly added features:
# Dimension were bounded now.
# Cell Type id added. (smallest available id)
# Advanced potts core setting added.

# Stretch goal:
# Try the Sorting Notebook demo, start looking at energy and initializers.


In [24]:
from Spec_UI import SpecificationSetupUI

ui = SpecificationSetupUI()

In [17]:
from cc3d.core.PyCoreSpecs import BlobInitializer

specs = [
    ui.metadata,
    ui.potts_core,
    ui.cell_type_plugin
]

cell_types = [ui.cell_type_plugin.cell_types[1], ui.cell_type_plugin.cell_types[2]]

blob_init_specs = BlobInitializer()
blob_init_specs.region_new(width=5, radius=20, center=(50, 50, 0), cell_types=cell_types)
specs.append(blob_init_specs)

print(cell_types)

['Condensing', 'NonCondensing']


In [ ]:

from cc3d.core.PyCoreSpecs import ContactPlugin

contact_specs = ContactPlugin(2)
contact_specs.param_new(type_1="Medium", type_2=cell_types[0], energy=16)
contact_specs.param_new(type_1="Medium", type_2=cell_types[1], energy=16)
contact_specs.param_new(type_1=cell_types[0], type_2=cell_types[0], energy=2)
contact_specs.param_new(type_1=cell_types[0], type_2=cell_types[1], energy=11)
contact_specs.param_new(type_1=cell_types[1], type_2=cell_types[1], energy=16)
specs.append(contact_specs)


In [19]:
from cc3d.CompuCellSetup.CC3DCaller import CC3DSimService

cc3d_sim = CC3DSimService()
cc3d_sim.register_specs(specs)
cc3d_sim.run()
cc3d_sim.init()
cc3d_sim.start()

WILL RUN SIMULATION FROM BEGINNING


a:\conda_envs\cc3d_env\lib\site-packages\cc3d\core\Configuration\SettingUtils.py:248: UserWarning: Setting not set: PlayerSizes
  warnings.warn(f'Setting not set: {_el.attrib["Name"]}')
a:\conda_envs\cc3d_env\lib\site-packages\cc3d\core\Configuration\SettingUtils.py:248: UserWarning: Setting not set: PlayerSizesFloating
  warnings.warn(f'Setting not set: {_el.attrib["Name"]}')
a:\conda_envs\cc3d_env\lib\site-packages\cc3d\core\Configuration\SettingUtils.py:248: UserWarning: Setting not set: RecentSimulations
  warnings.warn(f'Setting not set: {_el.attrib["Name"]}')


True

In [20]:
from IPython.display import display

cc3d_sim.visualize().show()
display(cc3d_sim.jupyter_run_button())

CC3DViewInteractiveWidget(height=600, layout=Layout(height='auto', width='100%'), width=600)

ToggleButton(value=False, description='Run Simulation')

In [ ]:
# Testing vertical tabs
import ipywidgets as widgets
from IPython.display import display

# Create content for each tab
out1 = widgets.Output()
out2 = widgets.Output()
out3 = widgets.Output()

with out1:
    print("Content of Tab 1")

with out2:
    print("Content of Tab 2")

with out3:
    print("Content of Tab 3")

# Create individual buttons instead of ToggleButtons for more control
tab1_btn = widgets.Button(description='Tab 1', layout=widgets.Layout(width='100%'))
tab2_btn = widgets.Button(description='Tab 2', layout=widgets.Layout(width='100%'))
tab3_btn = widgets.Button(description='Tab 3', layout=widgets.Layout(width='100%'))

# Put them into a VBox so they stack tightly
tabs_box = widgets.VBox([tab1_btn, tab2_btn, tab3_btn], layout=widgets.Layout(width='120px'))

# Output area
content_area = widgets.Output()

# Function to switch content
def show_tab(tab):
    content_area.clear_output()
    with content_area:
        if tab == 1:
            display(out1)
        elif tab == 2:
            display(out2)
        elif tab == 3:
            display(out3)

# Attach callbacks
tab1_btn.on_click(lambda b: show_tab(1))
tab2_btn.on_click(lambda b: show_tab(2))
tab3_btn.on_click(lambda b: show_tab(3))

# Layout the tabs and content
ui = widgets.HBox([tabs_box, content_area])

# Display
display(ui)

# Initialize first tab
show_tab(1)


In [5]:
import ipywidgets as widgets
from IPython.display import display
from cc3d.core.PyCoreSpecs import PLUGINS

for plugin_cls in PLUGINS:
    plugin = plugin_cls()
    print(f"Plugin: {plugin_cls.__name__}")
    for param_name in plugin.spec_dict.keys():
        # For demonstration, create a text box for each parameter
        value = plugin.spec_dict[param_name]
        input_widget = widgets.Text(
            value=str(value),
            description=param_name
        )
        display(input_widget)

Plugin: AdhesionFlexPlugin


Text(value='1', description='neighbor_order')

Text(value='[]', description='molecules')

Text(value='{}', description='densities')

Text(value='{}', description='binding_formulas')

...

Text(value='{}', description='field_specs')

Plugin: SurfacePlugin


Text(value='{}', description='params')

Plugin: VolumePlugin


Text(value='{}', description='params')

In [6]:
from cc3d.core.PyCoreSpecs import PLUGINS
print([p.__name__ for p in PLUGINS])

['AdhesionFlexPlugin', 'BoundaryPixelTrackerPlugin', 'CellTypePlugin', 'CenterOfMassPlugin', 'ChemotaxisPlugin', 'ConnectivityGlobalPlugin', 'ConnectivityPlugin', 'ContactPlugin', 'CurvaturePlugin', 'ExternalPotentialPlugin', 'FocalPointPlasticityPlugin', 'LengthConstraintPlugin', 'MomentOfInertiaPlugin', 'NeighborTrackerPlugin', 'PixelTrackerPlugin', 'SecretionPlugin', 'SurfacePlugin', 'VolumePlugin']
